# LSTM Network

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau 
from tensorflow.keras.layers import Dense, Dropout, LSTM,Conv1D, TimeDistributed, Flatten, SpatialDropout1D, GlobalAveragePooling1D, SimpleRNN, MaxPooling1D, GlobalMaxPooling1D, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, f1_score, recall_score, accuracy_score, roc_auc_score, confusion_matrix, precision_score, classification_report
from numpy import sqrt
from numpy import argmax
from matplotlib import pyplot as plt
from sklearn.utils import shuffle
import seaborn as sns
from numpy import genfromtxt

/Users/niall/opt/anaconda3/envs/DCUML/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/niall/opt/anaconda3/envs/DCUML/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/niall/opt/anaconda3/envs/DCUML/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/niall/opt/anaconda3/envs/DCUML

# Read Data

In [2]:
input_file = '/Users/niall/Documents/Cypress_Code/Data/AWGN/noise_data_awgn_snrfive_final.csv'
my_data_flipped = genfromtxt(input_file, delimiter=',')

OSError: /Users/niall/Documents/Cypress_Code/Data/AWGN/noise_data_awgn_snrfive_final.csv not found.

In [ ]:
new_data = np.delete(my_data_flipped, (0), axis=0)

In [ ]:
X2 = np.delete(new_data, (0,32), axis=1)

In [ ]:
Y2 = new_data[:,32]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X2, Y2, test_size=0.25, random_state=42)

In [ ]:
X_train = np.array(X_train).reshape((np.shape(X_train)[0], 1, np.shape(X_train)[1]))
X_test = np.array(X_test).reshape((np.shape(X_test)[0], 1, np.shape(X_test)[1]))

# 1. Build LSTM Network

In [14]:
num_neurons = 31
num_features = 31


model = Sequential()
model.add(LSTM(num_neurons, input_shape=(1, num_features), return_sequences=True))
model.add(LSTM(num_neurons, input_shape=(1, num_features), return_sequences=False))
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


In [15]:
model.summary()

print("inputs: " , model.input_shape)
print("outputs: ", model.output_shape)
print("actual inputs: ", np.shape(X_train))
print("actual outputs: ", np.shape(y_train))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 1, 31)             7812      
_________________________________________________________________
lstm_1 (LSTM)                (None, 31)                7812      
_________________________________________________________________
dense (Dense)                (None, 1)                 32        
Total params: 15,656
Trainable params: 15,656
Non-trainable params: 0
_________________________________________________________________
inputs:  (None, 1, 31)
outputs:  (None, 1)
actual inputs:  (225000, 1, 31)
actual outputs:  (225000,)


In [16]:
# opt = SGD(lr=0.05)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [17]:
print('Training LSTM...')

batch_size = 5

history = model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=5,
          validation_data=(X_test, y_test))

score, acc = model.evaluate(X_test, y_test, batch_size=batch_size)

Training LSTM...
Train on 225000 samples, validate on 75000 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/5
225000/225000 [==============================] - 272s 1ms/sample - loss: 0.0603 - acc: 0.9742 - val_loss: 0.0364 - val_acc: 0.9846
Epoch 2/5
225000/225000 [==============================] - 246s 1ms/sample - loss: 0.0321 - acc: 0.9860 - val_loss: 0.0322 - val_acc: 0.9860
Epoch 3/5
225000/225000 [==============================] - 295s 1ms/sample - loss: 0.0303 - acc: 0.9866 - val_loss: 0.0315 - val_acc: 0.9861
Epoch 4/5
225000/225000 [==============================] - 258s 1ms/sample - loss: 0.0291 - acc: 0.9869 - val_loss: 0.0333 - val_acc: 0.9856
Epoch 5/5
75000/75000 [==============================] - 22s 287us/sample - loss: 0.0318 - acc: 0.9859TA: 0s - loss: 0.03


In [18]:
print('score:', score)
print('accuracy:', acc)

score: 0.03184376627553008
accuracy: 0.9859333


In [38]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [32]:
# Let's save the model to disc
json = model.to_json()
with open("/Users/niall/Documents/Cypress_Code/Models/AWGN_Models/model_one_snr5.json", "w") as file:
    file.write(json)
model.save_weights("/Users/niall/Documents/Cypress_Code/Models/AWGN_Models/model_one_snr5.h5")
print("Saved model to disc")

Saved model to disc


In [33]:
# Load the model of interest
json_file = open('/Users/niall/Documents/Cypress_Code/Models/AWGN_Models/model_one_snr5.json', 'r')
json = json_file.read()
json_file.close()
model_one_from_disc = model_from_json(json)
model_one_from_disc.load_weights("/Users/niall/Documents/Cypress_Code/Models/AWGN_Models/model_one_snr5.h5")

In [30]:
y_pred_from_disk = model_one_from_disc.predict(X_test, batch_size=1, verbose=1)

75000/75000 [==============================] - 80s 1ms/sample


In [34]:
pred = pd.DataFrame(y_pred_from_disk, columns =['Y_pred']) 
pred['Label'] = y_test

In [36]:
pred.to_csv('/Users/niall/Documents/Cypress_Code/Predictions/AWGN/model_one_snr5.csv', index = False)

In [39]:
pred.head()

Y_pred  Label
0  9.993424e-01    1.0
1  9.999708e-01    1.0
2  4.346386e-08    0.0
3  7.989684e-06    0.0
4  6.149244e-02    0.0

In [42]:
pred.hist(column='Y_pred')

array([[<matplotlib.axes._subplots.AxesSubplot object at 0x7fde0e9f9690>]],
      dtype=object)

In [ ]:
## Predictions on Test Data

In [ ]:
def metrics(pred_tag, y_test):
    print("F1-score: ", f1_score(pred_tag, y_test))
    print("Precision: ", precision_score(pred_tag, y_test))
    print("Recall: ", recall_score(pred_tag, y_test))
    print("Acuracy: ", accuracy_score(pred_tag, y_test))
    print("-"*50)
    print(classification_report(pred_tag, y_test))
    
preds_new = model_one_from_disc.predict_classes(X_test)
metrics(preds_new, y_test)